<a href="https://colab.research.google.com/github/cheikh19/DaxStudio/blob/master/diabtes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=c1e8847a8ad922431670d94d7fce24713771e70b72c5b8b499251bc2a82871a3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Modele RF').getOrCreate()

# Chargez le fichier CSV dans un dataframe Spark
diatebtes = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/content/diatebtes.csv")

In [13]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

In [14]:
# Convertir les colonnes en vecteur
featureCols = diatebtes.columns[:-1]
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
diatebtes = assembler.transform(diatebtes).select(col("features"), col("Outcome").alias("label"))

AnalysisException: ignored

In [15]:
# Diviser les données en ensembles d'entraînement et de test
train_diabete, test_diabete = diatebtes.randomSplit([0.7, 0.3], seed=42)

In [16]:
# Créer le modèle
rf = RandomForestClassifier(numTrees=10, maxDepth=5, seed=42)

In [17]:
# Entraîner le modèle
model = rf.fit(train_diabete)

IllegalArgumentException: ignored

In [ ]:
# Évaluer le modèle
evaluator = BinaryClassificationEvaluator()
predictions = model.transform(test_diabete)

In [ ]:
auc = evaluator.evaluate(predictions)
print("AUC = ", auc)

AUC =  0.8055818994716163


In [ ]:
# Faire des prédictions sur de nouvelles données
new_entrees_diabetes = spark.createDataFrame([(0, 179, 50, 36, 159, 37.8, 0.445, 22)], featureCols)
new_entrees_diabetes = assembler.transform(new_entrees_diabetes)
prediction = model.predict(new_entrees_diabetes.first().features)
print("Prediction = ", prediction)

Prediction =  1.0
